In [1]:
import tensorflow as tf
import jpeg_related_functions as jrf
from PIL import Image
import multilayerCSC_ADMM as mlcsc

In [2]:
import numpy as np
q = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
              [12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],
              [14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],
              [24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],
              [72, 92, 95, 98, 112, 100, 103, 99]])
q = q.astype('float64')/255.
qY = q
qUV = q

In [3]:
image = Image.open("20190728_matthewdruincom_SLE_0033.jpg")
s = np.asarray(image).astype('float64')/255.
s = s[slice(1006,1038),slice(1352,1384),slice(None)]
croppedImage = Image.fromarray((255.*s).astype('uint8'),'RGB')
croppedImage.show()
print(s.shape)

(32, 32, 3)


In [4]:
rho = 0.1
alpha = 1.5
noi = 15
lmbda = 1.
fftSz = s.shape[0:2]
smooth_jpeg = jrf.Smooth_JPEG(rho,alpha,noi,qY,qUV,lmbda,fftSz,dtype=tf.float64)

In [5]:
lowpass,jpeg_image = smooth_jpeg(np.reshape(s,(1,) + s.shape))

In [6]:
y_new = tf.where(lowpass < 0,0.,lowpass)
y_new = tf.where(y_new > 1.0,1.0,y_new)
smoothImage = Image.fromarray((255.*np.asarray(tf.reshape(y_new,y_new.shape[1:]))).astype('uint8'),'RGB')
smoothImage.show()

In [7]:
x_new = tf.where(smooth_jpeg.Wt(jpeg_image) < 0.,0.,smooth_jpeg.Wt(jpeg_image))
x_new = tf.where(x_new > 1.0,1.0,x_new)
jpegImage = Image.fromarray((255.*np.asarray(tf.reshape(x_new,x_new.shape[1:]))).astype('uint8'),'RGB')
jpegImage.show()

In [8]:
highpass = smooth_jpeg.Wt(jpeg_image) - lowpass

In [9]:
z_new = tf.where(highpass < 0,0.,highpass)
z_new = tf.where(z_new > 1.0,1.0,z_new)
jpegImage = Image.fromarray((255.*np.asarray(tf.reshape(z_new,z_new.shape[1:]))).astype('uint8'),'RGB')
jpegImage.show()
jpegImage2 = Image.fromarray((-255.*np.asarray(tf.reshape(z_new,z_new.shape[1:]))).astype('uint8'),'RGB')
jpegImage2.show()

In [10]:
rho = 1.
alpha_init = 1.5
mu_init = 1.
fltrSz = ((3,3),(3,3))
noc = (3,8)
nof = (8,16)
lraParam = {'n_components': 3}
noi = 3
noL = 2
cmplxdtype = tf.complex128
CSC = mlcsc.MultiLayerCSC(rho,alpha_init,mu_init,qY,qUV,fltrSz,(fftSz,fftSz),noc,nof,lraParam,noi,noL,cmplxdtype)

In [11]:
x, itstats = CSC((lowpass,highpass,jpeg_image))

In [11]:
s = CSC.preprocess((lowpass,highpass,jpeg_image))
y,u,By,negC,itstats = CSC.init_vars(s)
x,Ax = CSC.xstep(y,u,By,negC)

In [220]:
Ax_relaxed = CSC.relax(Ax,By,negC)

In [221]:
beforeyupdate = CSC.evaluateRelaxedLagrangian(x,y,u,Ax_relaxed,By,negC)
#beforeyupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
y,By = CSC.ystep(x,u,Ax_relaxed,negC)
#afteryupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
afteryupdate = CSC.evaluateRelaxedLagrangian(x,y,u,Ax_relaxed,By,negC)

print('y-step improvement: ',beforeyupdate - afteryupdate)

y-step improvement:  tf.Tensor(2.8768026449496915, shape=(), dtype=float64)


In [222]:
u = CSC.ustep(u,Ax_relaxed,By,negC)

In [223]:
beforexupdate = CSC.evaluateLagrangian(x,y,u,By,negC)
x,Ax = CSC.xstep(y,u,By,negC)
afterxupdate = CSC.evaluateLagrangian(x,y,u,By,negC)

print('x-step improvement: ',beforexupdate - afterxupdate)

x-step improvement:  tf.Tensor(2.504578839214304, shape=(), dtype=float64)


In [84]:
print(x[0].dtype)

<dtype: 'complex128'>


In [15]:
print(x[0].dtype.real_dtype)

<dtype: 'float64'>


In [284]:
print(tf.reduce_max(tf.reshape(CSC.dictObj[0].R,(1,1,1,8,1))*CSC.IFFT[0](x[0])))

tf.Tensor(0.3018569557058902, shape=(), dtype=float64)


In [285]:
print(tf.reduce_min(CSC.IFFT[0](x[0])))

tf.Tensor(-0.018461204382282353, shape=(), dtype=float64)


In [286]:
print(tf.reduce_max(CSC.IFFT[0](y[0])))

tf.Tensor(1.2742018990043937, shape=(), dtype=float64)


In [287]:
print(tf.reduce_max(highpass))

tf.Tensor(0.2395837879066285, shape=(), dtype=float64)


In [348]:
print(tf.reduce_min(CSC.IFFT[0](y[0])))

tf.Tensor(-9.481495837540637, shape=(), dtype=float64)


In [351]:
print(np.amax(highpass))

0.30236045126053435


In [350]:
print([]*4)

[]


In [12]:
print([[],]*4)

[[], [], [], []]


In [13]:
tf.dtypes('complex128')

TypeError: 'module' object is not callable